In [ ]:
from dolfin import *
from mshr import *
from rbnics.backends.dolfin.wrapping import counterclockwise
from rbnics.shape_parametrization.utils.symbolic import VerticesMappingIO

In [ ]:
# Create domain
domain = Rectangle(Point(0., 0.), Point(1., 0.5))

In [ ]:
# Vertices mappings

vertices_mappings = [
    {
        ("0.0", "0.0"):("0.0", "0.0"),
        ("0.2", "0.0"):("mu[0]", "0.0"),
        ("0.0", "0.5"):("0.0", "0.5")
    }, # subdomain 1
    {
        ("0.0", "0.5"):("0.0", "0.5"),
        ("0.2", "0.0"):("mu[0]", "0.0"),
        ("1.0", "0.5"):("1.0", "0.5")
    }, # subdomain 2
    {
        ("0.2", "0.0"):("mu[0]", "0.0"),
        ("0.8", "0.0"):("1-mu[0]", "0.0"),
        ("1.0", "0.5"):("1.0", "0.5")
    }, # subdomain 3
    {
        ("0.8", "0.0"):("1-mu[0]", "0.0"),
        ("1.0","0.0"):("1.0","0.0"),
        ("1.0", "0.5"):("1.0", "0.5")
    } # subdomain 4
]

In [ ]:
for i, vertices_mapping in enumerate(vertices_mappings):
    subdomain_i = Polygon([Point(*[float(coord) for coord in vertex]) for vertex in counterclockwise(vertices_mapping.keys())])
    domain.set_subdomain(i + 1, subdomain_i)
mesh = generate_mesh(domain, 40)
plot(mesh)

In [ ]:
# Create subdomains
subdomains = MeshFunction("size_t", mesh, 2, mesh.domains())
plot(subdomains)

In [ ]:
# Create boundaries
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 0.) < DOLFIN_EPS

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 1.) < DOLFIN_EPS

class Bottom(SubDomain):
    def __init__(self, x_min, x_max):
        SubDomain.__init__(self)
        self.x_min = x_min
        self.x_max = x_max
        
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - 0.) < DOLFIN_EPS and x[0] >= self.x_min and x[0] <= self.x_max
                
class Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - 0.5) < DOLFIN_EPS 
        
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
bottomLeft = Bottom(0., 0.2)
bottomLeft.mark(boundaries, 1)
bottomMiddle = Bottom(0.2, 0.8)
bottomMiddle.mark(boundaries,2)
bottomRight = Bottom(0.8, 1.)
bottomRight.mark(boundaries, 3)
right = Right()
right.mark(boundaries, 4)
top = Top()
top.mark(boundaries,5)
left = Left()
left.mark(boundaries, 6)

In [ ]:
# Save
VerticesMappingIO.save_file(vertices_mappings, ".", "vertices_mapping.vmp")
File("friction.xml") << mesh
File("friction_physical_region.xml") << subdomains
File("friction_facet_region.xml") << boundaries
XDMFFile("friction.xdmf").write(mesh)
XDMFFile("friction_physical_region.xdmf").write(subdomains)
XDMFFile("friction_facet_region.xdmf").write(boundaries)